In [1]:
import sys
from pathlib import Path

# Get the absolute path to the project root
notebook_path = Path('/Users/nirmal/Documents/np_research/ManGo_code/ManGo')
project_root = str(notebook_path)

# Add the project root to Python path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Now you can import your module
from src.models.mg_rig_wrp import GnosticRobustRegressor

# Verify the import worked
# print(f"Project root: {project_root}")
# print(f"Available at: {RobustRegressor.__module__}")

/Users/nirmal/Documents/np_research/ManGo_code/ManGo/.venv/lib/python3.11/site-packages/mlflow/pyfunc/utils/data_validation.py:155: FutureWarning: Model's `predict` method contains invalid parameters: {'X'}. Only the following parameter names are allowed: context, model_input, and params. Note that invalid parameters will no longer be permitted in future versions.
  param_names = _check_func_signature(func, "predict")
/Users/nirmal/Documents/np_research/ManGo_code/ManGo/.venv/lib/python3.11/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
/Users/nirmal/Documents/np_research/ManGo_code/ManGo/.venv/lib/python3.11/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method t

In [9]:
import pandas as pd
import mlflow.pyfunc
from mlflow.models.signature import infer_signature

# Create and train the model
model = RobustRegressor(degree=3, mg_loss='hi', verbose=True)
X = np.random.rand(100, 2)
y = np.random.rand(100)
model.fit(X, y)

# Create an input example
input_example = pd.DataFrame(X, columns=["feature1", "feature2"])

# Infer the model signature
signature = infer_signature(input_example, model.predict( input_example))

# Log the model to MLflow with input_example and signature
with mlflow.start_run():
    mlflow.pyfunc.log_model(
        artifact_path="robust_regressor",
        python_model=model,
        registered_model_name="GnosticRobustRegressor",
        input_example=input_example,
        signature=signature
    )

2025/05/18 11:30:27 INFO mlflow.pyfunc: Validating input example against model signature


Machine Gnostic loss - hi : 74.8942
Machine Gnostic loss - hi : 84.8148


Registered model 'GnosticRobustRegressor' already exists. Creating a new version of this model...
Created version '2' of model 'GnosticRobustRegressor'.


In [7]:
# Load the model from MLflow
loaded_model = mlflow.pyfunc.load_model("models:/GnosticRobustRegressor/1")

# Use the model for predictions
import pandas as pd
X_test = pd.DataFrame(np.random.rand(10, 2))
predictions = loaded_model.predict(X_test)
print(predictions)

[0.45263084 0.31250219 0.2687311  0.2355985  0.36240116 0.31209806
 0.31387786 0.48452815 0.31547405 0.36282762]


In [13]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.pyfunc
from mlflow.models.signature import infer_signature
from src.models.mg_rig import RobustRegressor  # Replace with your actual import path

# Step 1: Set up MLflow experiment
mlflow.set_experiment("Robust Regressor Experiment")

# Step 2: Generate synthetic data
X = np.random.rand(100, 2)  # 100 samples, 2 features
y = 2.5 * X[:, 0] + 1.5 * X[:, 1] + np.random.normal(scale=0.1, size=100)  # Linear relationship with noise

# Step 3: Train the model
model = RobustRegressor(degree=3, mg_loss='hi', verbose=True)
model.fit(X, y)

# Step 4: Create an input example and infer the model signature
input_example = pd.DataFrame(X, columns=["feature1", "feature2"])
signature = infer_signature(input_example, model.predict(input_example))

# Step 5: Log the model, parameters, and metrics to MLflow
with mlflow.start_run():
    # Log parameters
    mlflow.log_param("degree", 3)
    mlflow.log_param("mg_loss", "hi")
    mlflow.log_param("verbose", True)

    # Log metrics (example: training RMSE)
    y_pred = model.predict(input_example)
    rmse = np.sqrt(np.mean((y - y_pred) ** 2))
    mlflow.log_metric("train_rmse", rmse)

    # Log the model
    mlflow.pyfunc.log_model(
        artifact_path="robust_regressor",
        python_model=model,
        registered_model_name="GnosticRobustRegressor",
        input_example=input_example,
        signature=signature
    )

# Step 6: Load the model from MLflow
loaded_model = mlflow.pyfunc.load_model("models:/GnosticRobustRegressor/3")

# Step 7: Use the loaded model for predictions
X_test = pd.DataFrame(np.random.rand(10, 2), columns=["feature1", "feature2"])
predictions = loaded_model.predict(X_test)
print("Predictions:")
print(predictions)

# Step 8: Serve the model as a REST API (optional)
# Run this command in the terminal to serve the model:
# mlflow models serve -m "models:/GnosticRobustRegressor/1" --port 5000
# Then, make predictions via REST API:
# curl -X POST -H "Content-Type: application/json" --data '{"columns":["feature1", "feature2"], "data":[[0.1, 0.2], [0.3, 0.4]]}' http://127.0.0.1:5000/invocations

2025/05/18 11:32:29 INFO mlflow.pyfunc: Validating input example against model signature


Machine Gnostic loss - hi : 84.4595
Machine Gnostic loss - hi : 84.8035
Predictions:
[1.40952163 2.94511034 2.19371197 3.53122398 0.67267672 3.22977985
 1.87467508 1.64238736 2.08641667 1.17793143]


Registered model 'GnosticRobustRegressor' already exists. Creating a new version of this model...
Created version '4' of model 'GnosticRobustRegressor'.
